# 🚀 Red Social CUDA - Google Colab

Este notebook te permite ejecutar tu proyecto de Red Social con CUDA en Google Colab usando GPUs NVIDIA gratuitas.

## ⚠️ IMPORTANTE: Habilitar GPU
1. Ve a **Runtime → Change runtime type**
2. En "Hardware accelerator" selecciona **GPU**
3. Haz clic en **Save**

## 📦 Paso 1: Verificar GPU disponible

In [ ]:
# Verificar que tenemos GPU disponible
!nvidia-smi

## 📂 Paso 2: Subir archivos del proyecto

Tienes dos opciones:

### Opción A: Subir archivos manualmente
Usa el botón de la izquierda (carpeta) para subir:
- `social_network.cu`
- `cuda_wrapper.py`

### Opción B: Conectar con Google Drive (recomendado)

In [ ]:
# Opción B: Montar Google Drive (descomenta si prefieres esta opción)
# from google.colab import drive
# drive.mount('/content/drive')

# Luego copia tus archivos desde Drive:
# !cp /content/drive/MyDrive/tu_carpeta/social_network.cu .
# !cp /content/drive/MyDrive/tu_carpeta/cuda_wrapper.py .

## 🔧 Paso 3: Instalar dependencias de Python

In [ ]:
!pip install streamlit pandas plotly

## 🛠️ Paso 4: Compilar código CUDA

In [ ]:
# Compilar el código CUDA
!nvcc -o social_network social_network.cu -std=c++11

## ▶️ Paso 5: Ejecutar el programa CUDA directamente

In [ ]:
# Ejecutar el programa CUDA
!./social_network

## 🐍 Paso 6: Usar el wrapper de Python (opcional)

In [ ]:
# Importar el wrapper
from cuda_wrapper import CUDASocialNetwork
import json

# Crear instancia (el ejecutable ya está compilado)
network = CUDASocialNetwork(executable="social_network")
network.compiled = True  # Ya lo compilamos manualmente

# Obtener datos parseados
data = network.get_parsed_data()

if data:
    print("✓ Datos parseados exitosamente")
    print(json.dumps(data, indent=2, ensure_ascii=False))
else:
    print("✗ Error al parsear datos")

## 📊 Paso 7: Visualización con Pandas

In [ ]:
import pandas as pd
import plotly.express as px

# Obtener datos
if data:
    # Crear DataFrame de seguidores de personas
    personas_df = pd.DataFrame(data['seguidores']['personas'])
    print("\n=== PERSONAS Y SEGUIDORES ===")
    print(personas_df.sort_values('seguidores', ascending=False))
    
    # Crear DataFrame de empresas
    empresas_df = pd.DataFrame(data['seguidores']['empresas'])
    print("\n=== EMPRESAS Y SEGUIDORES ===")
    print(empresas_df.sort_values('seguidores', ascending=False))
    
    # Crear DataFrame de reacciones
    reacciones_df = pd.DataFrame(data['reacciones'])
    print("\n=== REACCIONES POR POST ===")
    print(reacciones_df)
    
    # Crear DataFrame de hashtags
    hashtags_df = pd.DataFrame(data['hashtags']['conteo'])
    print("\n=== HASHTAGS ===")
    print(hashtags_df.sort_values('cantidad', ascending=False))

## 📈 Paso 8: Crear gráficos interactivos

In [ ]:
# Gráfico de personas por seguidores
if not personas_df.empty:
    fig = px.bar(
        personas_df.sort_values('seguidores', ascending=False).head(10),
        x='nombre',
        y='seguidores',
        title='Top 10 Personas por Seguidores',
        color='seguidores',
        color_continuous_scale='Blues'
    )
    fig.show()

In [ ]:
# Gráfico de empresas por seguidores
if not empresas_df.empty:
    fig = px.bar(
        empresas_df.sort_values('seguidores', ascending=False),
        x='nombre',
        y='seguidores',
        title='Empresas por Seguidores',
        color='seguidores',
        color_continuous_scale='Greens'
    )
    fig.show()

In [ ]:
# Gráfico de hashtags
if not hashtags_df.empty:
    fig = px.pie(
        hashtags_df,
        values='cantidad',
        names='hashtag',
        title='Distribución de Hashtags'
    )
    fig.show()

In [ ]:
# Gráfico de reacciones (Likes vs Dislikes)
if not reacciones_df.empty:
    import plotly.graph_objects as go
    
    fig = go.Figure()
    fig.add_trace(go.Bar(
        name='Likes',
        x=reacciones_df['post_id'],
        y=reacciones_df['likes'],
        marker_color='green'
    ))
    fig.add_trace(go.Bar(
        name='Dislikes',
        x=reacciones_df['post_id'],
        y=reacciones_df['dislikes'],
        marker_color='red'
    ))
    fig.update_layout(
        title='Likes vs Dislikes por Publicación',
        barmode='group',
        xaxis_title='Post ID',
        yaxis_title='Cantidad'
    )
    fig.show()

## 🌐 Paso 9: Ejecutar Streamlit en Colab (opcional)

⚠️ Nota: Streamlit en Colab requiere usar túneles (ngrok o localtunnel)

In [ ]:
# Instalar localtunnel para exponer Streamlit
!npm install -g localtunnel

In [ ]:
# Crear el archivo app.py (copiar tu código)
# Necesitas subir tu app.py a Colab o copiarlo aquí

# Ejecutar Streamlit en segundo plano
import subprocess
import time
from threading import Thread

def run_streamlit():
    subprocess.run(["streamlit", "run", "app.py", "--server.port", "8501"])

# Iniciar Streamlit en thread separado
thread = Thread(target=run_streamlit)
thread.start()

# Esperar a que Streamlit inicie
time.sleep(5)

# Exponer con localtunnel
!lt --port 8501

## 💾 Paso 10: Descargar resultados

In [ ]:
# Guardar datos en formato JSON
if data:
    with open('resultados.json', 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2, ensure_ascii=False)
    
    print("✓ Resultados guardados en resultados.json")
    
    # Descargar el archivo
    from google.colab import files
    files.download('resultados.json')

## 📝 Notas adicionales

### Límites de Google Colab:
- **Sesión gratuita**: ~12 horas continuas
- **GPU gratuita**: NVIDIA T4 (16GB) típicamente
- Los archivos se borran al cerrar la sesión

### Tips:
1. Guarda tu trabajo en Google Drive regularmente
2. Si necesitas más tiempo, considera Colab Pro
3. Puedes ejecutar celdas con `Shift + Enter`
4. Para reiniciar: `Runtime → Restart runtime`

### Solución de problemas:
- Si `nvcc` no funciona: Verifica que seleccionaste GPU en Runtime
- Si falta memoria: Usa `Runtime → Factory reset runtime`
- Si la sesión expira: Vuelve a ejecutar desde el Paso 1